## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-31-08-42-28 +0000,bbc,Takeaways from the millions of newly released ...,https://www.bbc.com/news/articles/cevnmxyy4wjo...
1,2026-01-31-07-47-58 +0000,bbc,Ukrainians brace for -20C despite energy truce...,https://www.bbc.com/news/articles/c75xg17rr1xo...
2,2026-01-31-07-38-26 +0000,nypost,Coast Guard searching for survivors after comm...,https://nypost.com/2026/01/31/us-news/massachu...
3,2026-01-31-07-24-40 +0000,nypost,Skeletal remains ID’d as 19-year-old Florida m...,https://nypost.com/2026/01/31/us-news/florida-...
4,2026-01-31-07-06-44 +0000,bbc,Canada's Poilievre re-elected Conservative par...,https://www.bbc.com/news/articles/c14r1023yy6o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2444/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
119,trump,69
4,epstein,28
175,ice,24
463,fed,22
471,warsh,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
338,2026-01-30-11-18-12 +0000,cbc,Trump names former Fed official turned critic ...,https://www.cbc.ca/news/world/trump-federal-re...,170
230,2026-01-30-17-24-50 +0000,nypost,Gold and silver plunge after news Trump will n...,https://nypost.com/2026/01/30/business/gold-si...,165
156,2026-01-30-21-08-13 +0000,nyt,"Kevin Warsh, Trump’s Choice for Fed Chair Coul...",https://www.nytimes.com/2026/01/30/business/tr...,150
205,2026-01-30-19-01-53 +0000,nyt,Trump Picks Kevin Warsh as Next Fed Chair,https://www.nytimes.com/2026/01/30/us/politics...,148
254,2026-01-30-16-20-00 +0000,wsj,"Kevin Warsh, a central-bank insider-turned-cri...",https://www.wsj.com/economy/central-banking/th...,147


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
338,170,2026-01-30-11-18-12 +0000,cbc,Trump names former Fed official turned critic ...,https://www.cbc.ca/news/world/trump-federal-re...
7,87,2026-01-31-06-19-29 +0000,nypost,LA Olympics boss Casey Wasserman’s racy emails...,https://nypost.com/2026/01/31/us-news/la-olymp...
11,81,2026-01-31-05-07-00 +0000,wsj,The Senate passed a bipartisan deal Friday to ...,https://www.wsj.com/politics/policy/senate-set...
71,76,2026-01-31-01-25-43 +0000,bbc,Former CNN host Don Lemon charged in anti-ICE ...,https://www.bbc.com/news/articles/c24gm3dz36po...
347,56,2026-01-30-11-00-00 +0000,nypost,New $65K private school uses AI to teach stude...,https://nypost.com/2026/01/30/business/new-65k...
102,49,2026-01-30-23-30-03 +0000,bbc,Luigi Mangione will not face death penalty if ...,https://www.bbc.com/news/articles/c78egdg3r0po...
105,45,2026-01-30-23-12-35 +0000,nypost,Disney CEO Bob Iger plans to step down before ...,https://nypost.com/2026/01/30/business/disney-...
187,42,2026-01-30-19-56-00 +0000,wsj,Kevin Warsh’s nomination to lead the Federal R...,https://www.wsj.com/economy/central-banking/wh...
322,35,2026-01-30-12-30-00 +0000,wsj,Panama High Court Gives Trump a Win Over Canal...,https://www.wsj.com/business/logistics/panamas...
302,34,2026-01-30-13-42-41 +0000,cbc,Trump's latest tariff threat has targeted Cana...,https://www.cbc.ca/news/world/livestory/trump-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
